In [192]:
 # import necessary all library
import time
start = time.time()
import geopandas as gpd
import pandas as pd
from functools import reduce
import sys
'geopandas' in sys.modules
#!pip3 install matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import io
import pandas as pd
import networkx as nx
import osmnx as ox
import os 
from geopandas import GeoDataFrame, read_file
from geopandas.tools import overlay
import shapely.geometry as geom
import shapely.wkt
import numpy as np

In [107]:
# Read bike counter variables data
bdata=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/Eco_CA_Bike_data_circular_0.5_mile_buffer.csv')
bdata.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,pct_at_least_college_education,HH_density,Median_HH_income,population_density,Median Age,employment_density,Total Commuter,Bike Commuter,Percentage of Bike Commuter,bike commuter density
0,671,37.791160,-122.459440,POLYGON ((-122.45030116607526 37.7911177158698...,5.153850,0.000000,0.797617,0.036196,0.527663,0.0,...,90.298608,3503.892064,152302.54020,7519.105739,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292
1,1127,37.796165,-122.419490,POLYGON ((-122.4103506127677 37.79611961207228...,3.019345,0.002189,0.249510,0.165813,0.131352,0.0,...,82.603467,26537.520610,130355.30360,20802.530110,38.239858,19609.011290,11234.019440,197.352495,1.756740,251.277818
2,2415,37.774200,-122.436295,POLYGON ((-122.42715829014239 37.7741559374201...,4.642047,0.000000,0.687274,0.087677,0.155897,0.0,...,86.089770,17752.654100,142485.80360,29059.732560,34.641712,26625.663510,16134.096210,1351.356394,8.375780,1720.605996
3,2691,35.287736,-120.657826,POLYGON ((-120.64898426571617 35.2875646426652...,76.562372,0.008590,0.397234,0.009334,0.254896,0.0,...,84.855253,3235.253308,33837.50721,7407.807740,23.961926,7014.619758,3148.238608,400.995099,12.737125,510.564478
4,2692,35.287736,-120.657826,POLYGON ((-120.64898426571617 35.2875646426652...,76.562372,0.008590,0.397234,0.009334,0.254896,0.0,...,84.855253,3235.253308,33837.50721,7407.807740,23.961926,7014.619758,3148.238608,400.995099,12.737125,510.564478


In [196]:
bike_fac=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/Facilities_data/CA_Bike_Facilities- Bike_Manual_facilities.csv')
bike_fac=bike_fac.drop(['latitude','longitude'],axis=1)
bike_fac.head()

,Name,ID,name,highway,Bicycle_Fa
0,1,671,Bay Area Ridge Trail,Path,Path
1,2,1127,Broadway,primary,bike route
2,3,2415,Fell Street,primary,bike lane
3,4,2691,Halsey Circle,residential,Path
4,5,2692,Halsey Circle,residential,Path


In [197]:
bdata.shape, bike_fac.shape

((416, 46), (414, 5))

In [198]:
bdata1=pd.merge(bdata, bike_fac, on=['ID'], how='inner')
bdata1.shape

(414, 50)

In [199]:
# add our MADBT after QC
madbt=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/our_bike_AASHTO_MADT for CA.csv')
madbt.head()

,Unnamed: 0,ID,year,month,AASHTO_MADT
0,0,671,2018,1,65.478571
1,1,671,2018,2,60.928571
2,2,671,2018,3,48.907143
3,3,671,2018,4,52.942857
4,4,671,2018,5,64.071429


In [200]:
madbt.groupby(['year'])["ID"].nunique()

year
2018    175
2019    165
2020    142
2021    156
Name: ID, dtype: int64

In [201]:
madbt["ID"].nunique()

250

In [202]:
# Lets add the variable
madbt1=pd.merge(madbt, bdata1, on=['ID'], how='inner')
madbt1.head()

,Unnamed: 0,ID,year,month,AASHTO_MADT,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,...,Median Age,employment_density,Total Commuter,Bike Commuter,Percentage of Bike Commuter,bike commuter density,Name,name,highway,Bicycle_Fa
0,0,671,2018,1,65.478571,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292,1,Bay Area Ridge Trail,Path,Path
1,1,671,2018,2,60.928571,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292,1,Bay Area Ridge Trail,Path,Path
2,2,671,2018,3,48.907143,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292,1,Bay Area Ridge Trail,Path,Path
3,3,671,2018,4,52.942857,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292,1,Bay Area Ridge Trail,Path,Path
4,4,671,2018,5,64.071429,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292,1,Bay Area Ridge Trail,Path,Path


In [203]:
madbt1.shape

(5327, 54)

In [204]:
madbt1["ID"].nunique()

250

In [268]:
# Now add Strava MADBT data
stv=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/Eco_Strava_bike_data_18_21_May_8.csv')
cols=["year","month"]
stv['date'] = stv[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
stv['date'] = pd.to_datetime(stv['date'], errors='coerce')
stv['days_in_month'] = stv['date'].dt.days_in_month
stv['Strava_MADT']=stv['stv_total']/stv['days_in_month']
stv.head()

,Unnamed: 0,ID,latitude,longitude,edgeUID,osmId,edge_uid,osmId.1,month,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,year,date,days_in_month,Strava_MADT
0,0,671,37.79116,-122.45944,308723101.0,215845474.0,308723101,215845474.0,8,420,20,400,0.047619,0.952381,2018,2018-08-01,31,13.548387
1,1,671,37.79116,-122.45944,308723101.0,215845474.0,308723101,215845474.0,12,385,10,375,0.025974,0.974026,2018,2018-12-01,31,12.419355
2,2,671,37.79116,-122.45944,308723101.0,215845474.0,308723101,215845474.0,1,395,25,370,0.063291,0.936709,2018,2018-01-01,31,12.741935
3,3,671,37.79116,-122.45944,308723101.0,215845474.0,308723101,215845474.0,11,270,15,255,0.055556,0.944444,2018,2018-11-01,30,9.000000
4,4,671,37.79116,-122.45944,308723101.0,215845474.0,308723101,215845474.0,4,430,30,400,0.069767,0.930233,2018,2018-04-01,30,14.333333


In [269]:
madbt2=pd.merge(madbt1, stv, on=["ID",'year','month','latitude','longitude'], how='inner')
madbt2.shape


(4540, 67)

In [270]:
madbt2['ID'].nunique()

181

In [271]:
# rest of ID have zero strava Activities
common = madbt1.merge(stv,on=["ID",'year','month','latitude','longitude'])
zero_stv=madbt1[(~madbt1.ID.isin(common.ID))&(~madbt1.ID.isin(common.ID))]
zero_stv.shape

(789, 54)

In [272]:
# Now concate the zero strava sites
madbt3=pd.concat([madbt2, zero_stv],axis=0)
madbt3=madbt3.fillna(0)
madbt3.head()

,Unnamed: 0_x,ID,year,month,AASHTO_MADT,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,...,osmId.1,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT,Unnamed: 0
0,0.0,671,2018,1,65.478571,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,215845474.0,395.0,25.0,370.0,0.063291,0.936709,2018-01-01 00:00:00,31.0,12.741935,0.0
1,1.0,671,2018,2,60.928571,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,215845474.0,465.0,25.0,440.0,0.053763,0.946237,2018-02-01 00:00:00,28.0,16.607143,0.0
2,2.0,671,2018,3,48.907143,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,215845474.0,385.0,10.0,375.0,0.025974,0.974026,2018-03-01 00:00:00,31.0,12.419355,0.0
3,3.0,671,2018,4,52.942857,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,215845474.0,430.0,30.0,400.0,0.069767,0.930233,2018-04-01 00:00:00,30.0,14.333333,0.0
4,4.0,671,2018,5,64.071429,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,215845474.0,475.0,25.0,450.0,0.052632,0.947368,2018-05-01 00:00:00,31.0,15.322581,0.0


In [273]:
madbt3['highway'].unique()

array(['Path', 'cycleway', 'secondary', 'tertiary', 'primary',
       'residential', 'service', 'pedestrian', 'trunk'], dtype=object)

In [274]:
madbt3['Path'] = np.where(madbt3['Bicycle_Fa']=='Path', 1, 0)
madbt3['bike lane'] = np.where(madbt3['Bicycle_Fa']=='bike lane', 1, 0)
madbt3['bike route'] = np.where(madbt3['Bicycle_Fa']=='bike route', 1, 0)
madbt3['Bike Lane'] = np.where(madbt3['Bicycle_Fa']=='Bike Lane', 1, 0)
madbt3['Cycle Track'] = np.where(madbt3['Bicycle_Fa']=='Cycle Track', 1, 0)
madbt3['Trail'] = np.where(madbt3['Bicycle_Fa']=='Trail', 1, 0)

In [275]:
madbt3['Path'] = np.where(madbt3['highway']=='Path', 1, 0)
madbt3['cycleway'] = np.where(madbt3['highway']=='cycleway', 1, 0)
madbt3['secondary'] = np.where(madbt3['highway']=='secondary', 1, 0)
madbt3['tertiary'] = np.where(madbt3['highway']=='tertiary', 1, 0)
madbt3['primary'] = np.where(madbt3['highway']=='primary', 1, 0)
madbt3['residential'] = np.where(madbt3['highway']=='residential', 1, 0)
madbt3['service'] = np.where(madbt3['highway']=='service', 1, 0)
madbt3['trunk'] = np.where(madbt3['highway']=='trunk', 1, 0)

In [290]:
madbt3.shape, madbt3['ID'].nunique()

((5329, 81), 250)

In [276]:
# import Imputation model data
madbt3.to_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/RF_Imputation_model_data/bike_imputation_model_data.csv')

# prepared bike model application data

In [277]:
all_data=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/all_valid_non_valid_bike_counters_data.csv')
non_valid_data=all_data.loc[all_data['days_in_month']>all_data['no_of_days_data_collected']]
non_valid_data=non_valid_data.drop_duplicates(subset=['ID','year','month'], keep='first')
non_valid_data=non_valid_data[['ID','year','month']]
non_valid_data.head(4)

,ID,year,month
151,671,2018,6
729,671,2020,1
972,671,2020,9
973,671,2020,11


In [278]:
non_valid_data['ID'].nunique()

390

In [292]:
# get the sites that was not qualified in QC
common = bdata.merge(madbt,on=['ID'])
nqcb=bdata[(~bdata.ID.isin(common.ID))&(~bdata.ID.isin(common.ID))]
nqcb1=pd.merge(nqcb, bike_fac, on=['ID'], how='inner')
nqcb1.shape

(164, 50)

In [293]:
nqcb2=pd.merge(nqcb1,non_valid_data,on=["ID"], how='inner')
nqcb2.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,Total Commuter,Bike Commuter,Percentage of Bike Commuter,bike commuter density,Name,name,highway,Bicycle_Fa,year,month
0,1127,37.796165,-122.419490,POLYGON ((-122.4103506127677 37.79611961207228...,3.019345,0.002189,0.249510,0.165813,0.131352,0.0,...,11234.019440,197.352495,1.756740,251.277818,2,Broadway,primary,bike route,2018,1
1,2415,37.774200,-122.436295,POLYGON ((-122.42715829014239 37.7741559374201...,4.642047,0.000000,0.687274,0.087677,0.155897,0.0,...,16134.096210,1351.356394,8.375780,1720.605996,3,Fell Street,primary,bike lane,2018,1
2,2691,35.287736,-120.657826,POLYGON ((-120.64898426571617 35.2875646426652...,76.562372,0.008590,0.397234,0.009334,0.254896,0.0,...,3148.238608,400.995099,12.737125,510.564478,4,Halsey Circle,residential,Path,2018,1
3,2692,35.287736,-120.657826,POLYGON ((-120.64898426571617 35.2875646426652...,76.562372,0.008590,0.397234,0.009334,0.254896,0.0,...,3148.238608,400.995099,12.737125,510.564478,5,Halsey Circle,residential,Path,2018,1
4,2696,33.636400,-117.573600,POLYGON ((-117.5649623942388 33.63601951720258...,30.272075,0.000000,4.698224,0.000000,5.178166,0.0,...,418.449730,0.000000,0.000000,0.000000,8,,cycleway,Trail,2018,1


In [294]:
nqcb2['ID'].nunique()

162

In [282]:
stv.columns

Index(['Unnamed: 0', 'ID', 'latitude', 'longitude', 'edgeUID', 'osmId',
       'edge_uid', 'osmId.1', 'month', 'stv_total', 'stv_com_total',
       'stv_rec_total', 'stv_com_ratio', 'stv_rec_ratio', 'year', 'date',
       'days_in_month', 'Strava_MADT'],
      dtype='object')

In [295]:
# Lets add Strava 
nqcb3=pd.merge(nqcb2,stv, on=['ID','latitude','longitude','year','month'], how='inner')
nqcb3.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,edge_uid,osmId.1,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT
0,1127,37.796165,-122.419490,POLYGON ((-122.4103506127677 37.79611961207228...,3.019345,0.002189,0.249510,0.165813,0.131352,0.000000,...,308779961,87297166.0,25,10,15,0.400000,0.600000,2018-01-01,31,0.806452
1,2415,37.774200,-122.436295,POLYGON ((-122.42715829014239 37.7741559374201...,4.642047,0.000000,0.687274,0.087677,0.155897,0.000000,...,308725144,907910631.0,2200,745,1455,0.338636,0.661364,2018-01-01,31,70.967742
2,2696,33.636400,-117.573600,POLYGON ((-117.5649623942388 33.63601951720258...,30.272075,0.000000,4.698224,0.000000,5.178166,0.000000,...,308341527,252712475.0,20,0,20,0.000000,1.000000,2018-01-01,31,0.645161
3,2735,37.851585,-122.260410,POLYGON ((-122.25126407202679 37.8515272002856...,3.516737,0.008472,0.261220,0.035638,0.030689,0.000676,...,309663180,416105082.0,220,85,135,0.386364,0.613636,2018-01-01,31,7.096774
4,3563,32.991085,-117.271470,POLYGON ((-117.26290023488116 32.9906902025036...,77.836301,0.004015,0.124383,0.014682,0.099479,0.000000,...,305965867,345780741.0,3715,215,3500,0.057873,0.942127,2018-01-01,31,119.838710


In [296]:
nqcb3['ID'].nunique()

85

In [297]:
# add the sites that have strava zero
# rest of ID have zero strava Activities
common = nqcb2.merge(nqcb3,on=['ID','latitude','longitude','year','month'])
zero_ap=nqcb2[(~nqcb2.ID.isin(common.ID))&(~nqcb2.ID.isin(common.ID))]
zero_ap.shape

(134, 52)

In [298]:
# add both
# Now concate the zero strava sites
nqcb4=pd.concat([nqcb3, zero_ap],axis=0)
nqcb4=nqcb4.fillna(0)
nqcb4.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,edge_uid,osmId.1,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT
0,1127,37.796165,-122.419490,POLYGON ((-122.4103506127677 37.79611961207228...,3.019345,0.002189,0.249510,0.165813,0.131352,0.000000,...,308779961.0,87297166.0,25.0,10.0,15.0,0.400000,0.600000,2018-01-01 00:00:00,31.0,0.806452
1,2415,37.774200,-122.436295,POLYGON ((-122.42715829014239 37.7741559374201...,4.642047,0.000000,0.687274,0.087677,0.155897,0.000000,...,308725144.0,907910631.0,2200.0,745.0,1455.0,0.338636,0.661364,2018-01-01 00:00:00,31.0,70.967742
2,2696,33.636400,-117.573600,POLYGON ((-117.5649623942388 33.63601951720258...,30.272075,0.000000,4.698224,0.000000,5.178166,0.000000,...,308341527.0,252712475.0,20.0,0.0,20.0,0.000000,1.000000,2018-01-01 00:00:00,31.0,0.645161
3,2735,37.851585,-122.260410,POLYGON ((-122.25126407202679 37.8515272002856...,3.516737,0.008472,0.261220,0.035638,0.030689,0.000676,...,309663180.0,416105082.0,220.0,85.0,135.0,0.386364,0.613636,2018-01-01 00:00:00,31.0,7.096774
4,3563,32.991085,-117.271470,POLYGON ((-117.26290023488116 32.9906902025036...,77.836301,0.004015,0.124383,0.014682,0.099479,0.000000,...,305965867.0,345780741.0,3715.0,215.0,3500.0,0.057873,0.942127,2018-01-01 00:00:00,31.0,119.838710


In [304]:
check=nqcb4.loc[nqcb4['ID']==58246]
check

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,edge_uid,osmId.1,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT
122,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,5.0,0.0,5.0,0.000000,1.000000,2018-01-01 00:00:00,31.0,0.161290
123,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,20.0,5.0,15.0,0.250000,0.750000,2019-03-01 00:00:00,31.0,0.645161
124,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,510.0,25.0,485.0,0.049020,0.950980,2019-04-01 00:00:00,30.0,17.000000
125,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,5.0,0.0,5.0,0.000000,1.000000,2019-05-01 00:00:00,31.0,0.161290
126,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,215.0,40.0,175.0,0.186047,0.813953,2019-06-01 00:00:00,30.0,7.166667
127,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,385.0,100.0,285.0,0.259740,0.740260,2019-10-01 00:00:00,31.0,12.419355
128,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,20.0,5.0,15.0,0.250000,0.750000,2019-11-01 00:00:00,30.0,0.666667
129,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,315.0,65.0,250.0,0.206349,0.793651,2019-12-01 00:00:00,31.0,10.161290


In [299]:
nqcb4['ID'].nunique()

162

In [300]:
nqcb4.columns

Index(['ID', 'latitude', 'longitude', 'geometry', 'min_dist_to_CBD',
       'Commercial Area ', 'Distance to commercial area', 'retail Area ',
       'Distance to retail area', 'industrial Area ',
       'Distance to industrial area', 'residential Area ',
       'Distance to residential area', 'grass Area ', 'Distance to grass area',
       'forest Area ', 'Distance to forest area', 'education Area ',
       'Distance to education area', 'institutional Area ',
       'Distance to institutional area', 'water Area',
       'Distance to water area', 'park Area ', 'Distance to Park Center',
       'School', 'min_dist_to_school', 'college', 'min_dist_to_college',
       'University', 'min_dist_to_university', 'pct_white',
       'pct_African_American', 'pct_male', 'pct_female', 'Student Access',
       'pct_at_least_college_education', 'HH_density', 'Median_HH_income',
       'population_density', 'Median Age', 'employment_density',
       'Total Commuter', 'Bike Commuter', 'Percentage of B

In [301]:
# import Imputation model data
nqcb3.to_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/RF_Imputation_model_data/bike_imputation_model_application_data.csv')

# Process Pedestrain data

In [3]:
# Read ped counter variables data
pdata=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/Eco_CA_Ped_data_circular_0.25_mile_buffer.csv')
pdata.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,pct_at_least_college_education,HH_density,Median_HH_income,population_density,Median Age,employment_density,Total Commuter,Walk Commuter,Percentage of Walk Commuter,Walk commuter density
0,572,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,1.001268,0.0,0.586396,0.0,...,91.947499,559.508075,208000.000000,1152.823284,30.200000,931.809142,138.319911,8.546461,6.178764,43.526911
1,588,37.793898,-122.455561,POLYGON ((-122.45099097999639 37.7938764145384...,4.889865,0.0,0.515058,0.0,0.655687,0.0,...,92.279192,592.390519,210739.958418,1297.035477,32.438325,1026.436451,149.147696,8.721225,5.847375,44.416978
2,589,37.793898,-122.455561,POLYGON ((-122.45099097999639 37.7938764145384...,4.889865,0.0,0.515058,0.0,0.655687,0.0,...,92.279192,592.390519,210739.958418,1297.035477,32.438325,1026.436451,149.147696,8.721225,5.847375,44.416978
3,595,37.792583,-122.458150,POLYGON ((-122.45358049399331 37.7925618958346...,5.053731,0.0,0.678132,0.0,0.721444,0.0,...,93.688703,846.104922,219501.087641,2392.231666,40.054783,1769.674912,235.847198,10.058299,4.264752,51.226661
4,671,37.791160,-122.459440,POLYGON ((-122.45487058064846 37.7911389465364...,5.153850,0.0,0.797617,0.0,0.818882,0.0,...,92.301012,1817.953749,182448.977408,5413.545187,40.152337,4314.354911,549.884362,19.808159,3.602241,100.882451


In [96]:
ped_fac=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/Facilities_data/CA_Ped_Facilities- Ped_manual_facilities.csv')
ped_fac=ped_fac.drop(['latitude','longitude'], axis=1)
ped_fac.head()

,Name,ID,Pedestrain Facilities,highway
0,1,572,Trail,Path
1,2,588,Path,residential
2,4,595,Trail,Path
3,5,671,Path,Path
4,6,780,Trail,Path


In [97]:
pdata1=pd.merge(pdata, ped_fac, on=['ID'], how='inner')
pdata1.shape

(282, 49)

In [98]:
# add our MADPT after QC
madpt=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/our_ped_AASHTO_MADT for CA.csv')
madpt.head()

,Unnamed: 0,ID,year,month,AASHTO_MADT
0,0,572,2018,1,188.971429
1,1,572,2018,2,213.142857
2,2,572,2018,3,235.550000
3,3,572,2018,4,221.250000
4,4,572,2018,5,220.435714


In [99]:
madpt.shape

(3922, 5)

In [100]:
madpt.groupby(['year'])["ID"].nunique()

year
2018     90
2019     88
2020     92
2021    123
2022    145
Name: ID, dtype: int64

In [101]:
madpt["ID"].nunique()

208

In [119]:
# Lets add the variable
madpt1=pd.merge(madpt, pdata1, on=['ID'], how='inner')
madpt1.head()

,Unnamed: 0,ID,year,month,AASHTO_MADT,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,...,population_density,Median Age,employment_density,Total Commuter,Walk Commuter,Percentage of Walk Commuter,Walk commuter density,Name,Pedestrain Facilities,highway
0,0,572,2018,1,188.971429,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,1152.823284,30.2,931.809142,138.319911,8.546461,6.178764,43.526911,1,Trail,Path
1,1,572,2018,2,213.142857,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,1152.823284,30.2,931.809142,138.319911,8.546461,6.178764,43.526911,1,Trail,Path
2,2,572,2018,3,235.550000,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,1152.823284,30.2,931.809142,138.319911,8.546461,6.178764,43.526911,1,Trail,Path
3,3,572,2018,4,221.250000,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,1152.823284,30.2,931.809142,138.319911,8.546461,6.178764,43.526911,1,Trail,Path
4,4,572,2018,5,220.435714,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,1152.823284,30.2,931.809142,138.319911,8.546461,6.178764,43.526911,1,Trail,Path


In [106]:
madpt1["ID"].nunique()

208

In [104]:
# get the sites that was not qualified in QC
common = pdata.merge(madpt,on=['ID'])
nqc=pdata[(~pdata.ID.isin(common.ID))&(~pdata.ID.isin(common.ID))]
nqc.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,pct_at_least_college_education,HH_density,Median_HH_income,population_density,Median Age,employment_density,Total Commuter,Walk Commuter,Percentage of Walk Commuter,Walk commuter density
1,588,37.793898,-122.455561,POLYGON ((-122.45099097999639 37.7938764145384...,4.889865,0.0,0.515058,0.0,0.655687,0.0,...,92.279192,592.390519,210739.958418,1297.035477,32.438325,1026.436451,149.147696,8.721225,5.847375,44.416978
2,589,37.793898,-122.455561,POLYGON ((-122.45099097999639 37.7938764145384...,4.889865,0.0,0.515058,0.0,0.655687,0.0,...,92.279192,592.390519,210739.958418,1297.035477,32.438325,1026.436451,149.147696,8.721225,5.847375,44.416978
5,780,37.798060,-122.467319,POLYGON ((-122.46274951901897 37.7980395200501...,5.423048,0.0,0.987122,0.0,0.573773,0.0,...,91.947499,559.508075,208000.000000,1152.823284,30.200000,931.809142,138.319911,8.546461,6.178764,43.526911
6,781,37.798060,-122.467319,POLYGON ((-122.46274951901897 37.7980395200501...,5.423048,0.0,0.987122,0.0,0.573773,0.0,...,91.947499,559.508075,208000.000000,1152.823284,30.200000,931.809142,138.319911,8.546461,6.178764,43.526911
11,2696,33.636400,-117.573600,POLYGON ((-117.56928117684645 33.6362098335502...,30.272075,0.0,4.980704,0.0,5.164795,0.0,...,92.068092,299.628122,213629.271604,786.796545,46.763146,644.715602,74.938867,0.000005,0.000007,0.000026


In [74]:
# Get Ped MADT data
ped_var=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/Strava Data/Ped_modeling_data.csv')
ped_var1=ped_var[['ID', 'year','month','days_in_month','edge_uid', 'stv_total', 'stv_com_total',
       'stv_rec_total', 'stv_com_ratio', 'stv_rec_ratio']]
ped_var1['Strava_MADT']=ped_var1['stv_total']/ped_var1['days_in_month']
ped_var1.head()

C:\Users\Md Mintu Miah\AppData\Local\Temp\ipykernel_26688\2458223688.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ped_var1['Strava_MADT']=ped_var1['stv_total']/ped_var1['days_in_month']


,ID,year,month,days_in_month,edge_uid,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,Strava_MADT
0,19211,2018,1,31,305679020,225,10,215,0.044444,0.955556,7.258065
1,19211,2018,2,28,305679020,135,10,125,0.074074,0.925926,4.821429
2,19211,2018,3,31,305679020,235,5,230,0.021277,0.978723,7.580645
3,19211,2018,4,30,305679020,200,15,185,0.075000,0.925000,6.666667
4,19211,2018,5,31,305679020,205,15,190,0.073171,0.926829,6.612903


In [75]:
ped_var['year'].unique()

array([2018, 2019, 2020, 2021, 2022], dtype=int64)

In [79]:
ped_var['ID'].nunique()

195

In [76]:
ped_data2=pd.merge(madpt1, ped_var1, on=["ID",'year','month'], how='inner')
ped_data2.head()

,Unnamed: 0,ID,year,month,AASHTO_MADT,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,...,Percentage of Walk Commuter,Walk commuter density,days_in_month,edge_uid,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,Strava_MADT
0,0,572,2018,1,188.971429,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,6.178764,43.526911,31,308772257,535,35,500,0.065421,0.934579,17.258065
1,1,572,2018,2,213.142857,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,6.178764,43.526911,28,308772257,520,50,470,0.096154,0.903846,18.571429
2,2,572,2018,3,235.550000,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,6.178764,43.526911,31,308772257,615,40,575,0.065041,0.934959,19.838710
3,3,572,2018,4,221.250000,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,6.178764,43.526911,30,308772257,545,50,495,0.091743,0.908257,18.166667
4,4,572,2018,5,220.435714,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,6.178764,43.526911,31,308772257,600,45,555,0.075000,0.925000,19.354839


In [78]:
ped_data2['ID'].nunique()

195

In [120]:
# import Imputation model data
madpt1.to_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/RF_Imputation_model_data/ped_imputation_model_data.csv')